# Config

In [3]:
# file_path = "/content/drive/MyDrive/mimic-iii-clinicalnote-v4.1/bq-results-20231215-074521-1702626519432.csv"
# output_folder_path = "/content/drive/MyDrive/mimic-iii-clinicalnote-v4.1/processed_set"

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
file_path = "../data/mimic-iii-clinicalnote-v4.1.csv"
output_folder_path = "../data/processed_set"

# import package and load the data

In [6]:
import pandas as pd
import numpy as np
import torch
import os
import string
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel

2024-01-18 15:03:49.904482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# load data

In [7]:
X_train = pd.read_csv(os.path.join(output_folder_path, 'X_train_raw.csv'))
X_test = pd.read_csv(os.path.join(output_folder_path, 'X_test_raw.csv'))
X_val = pd.read_csv(os.path.join(output_folder_path, 'X_val_raw.csv'))

X_train

,icustay_id,slice_start,avg_pCO2,avg_Glucose,avg_Platelet_Count,avg_Potassium,avg_Calculated_total_CO2,avg_Urea_nitrogen,avg_Respiratory_rate,avg_Heartrate,avg_Arterial_blood_pressure_mean,avg_Arterial_blood_pressure_systolic,avg_Central_venous_pressure,avg_Arterial_blood_pressure_diastolic,avg_Tidal_volume_obs,avg_Temp_C,avg_Height_cm,processed_text
0,200009,2189-11-30 10:34:32,-0.224090,-0.826832,-0.335981,-0.357971,1.004672,-0.379075,-0.003306,-0.019297,0.004217,0.013645,-0.017599,0.000598,-0.011438,0.032409,0.000066,missing
1,200009,2189-11-30 12:34:32,-0.899159,-0.775719,-0.335981,1.654167,1.004672,-0.379075,-0.003306,-0.019297,0.004217,0.013645,-0.017599,0.000598,-0.011438,0.032409,0.000066,missing
2,200009,2189-11-30 14:34:32,0.180951,-0.446325,-0.335981,2.263906,0.014136,-0.379075,-0.003306,-0.019297,0.004217,0.013645,-0.017599,0.000598,-0.011438,0.032409,0.000066,missing
3,200009,2189-11-30 16:34:32,-0.089076,-0.519343,-0.240436,0.449933,0.191017,-0.408397,-0.003306,-0.019297,0.004217,0.013645,-0.017599,0.000598,-0.011438,0.032409,0.000066,respiratory care note pt received intubated pl...
4,200009,2189-11-30 18:34:32,0.045937,-0.554230,-0.335981,-0.540892,0.261770,-0.379075,-0.003306,-0.019297,0.004217,0.013645,-0.017599,0.000598,-0.011438,0.032409,0.000066,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316303,299998,2181-07-06 08:47:40,-0.494118,0.558894,-0.206258,0.129820,-0.109681,-0.056537,-0.009762,-0.624710,-0.159017,2.338058,-0.501375,-0.155391,-0.117112,0.117644,0.000066,missing
316304,299998,2181-07-06 10:47:40,-0.494118,0.558894,-0.206258,0.129820,-0.109681,-0.056537,-0.009762,-0.668506,-0.367227,1.920538,-0.501375,-0.176385,-0.117112,0.101268,0.000066,missing
316305,299998,2181-07-06 12:47:40,-0.494118,0.740628,-0.221793,-0.357971,-0.109681,-0.056537,-0.007229,-0.274342,2.703869,1.642191,-0.501375,-0.176385,-0.117112,0.041222,0.000066,missing
316306,299998,2181-07-06 14:47:40,-0.494118,0.558894,-0.206258,0.129820,-0.109681,-0.056537,-0.010015,0.207414,3.068236,9.714239,-0.501375,0.068545,-0.117112,0.101268,0.000066,missing


In [8]:
X_train_clinicalbert = X_train[['icustay_id', 'slice_start', 'processed_text']]
X_test_clinicalbert = X_test[['icustay_id', 'slice_start', 'processed_text']]
X_val_clinicalbert = X_val[['icustay_id', 'slice_start', 'processed_text']]
X_train_clinicalbert

,icustay_id,slice_start,processed_text
0,200009,2189-11-30 10:34:32,missing
1,200009,2189-11-30 12:34:32,missing
2,200009,2189-11-30 14:34:32,missing
3,200009,2189-11-30 16:34:32,respiratory care note pt received intubated pl...
4,200009,2189-11-30 18:34:32,missing
...,...,...,...
316303,299998,2181-07-06 08:47:40,missing
316304,299998,2181-07-06 10:47:40,missing
316305,299998,2181-07-06 12:47:40,missing
316306,299998,2181-07-06 14:47:40,missing


In [9]:
X_train_clinicalbert['processed_text'] = X_train_clinicalbert['processed_text'].fillna('missing')
X_test_clinicalbert['processed_text'] = X_test_clinicalbert['processed_text'].fillna('missing')
X_val_clinicalbert['processed_text'] = X_val_clinicalbert['processed_text'].fillna('missing')
X_train_clinicalbert

/tmp/ipykernel_136663/905987518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_clinicalbert['processed_text'] = X_train_clinicalbert['processed_text'].fillna('missing')
/tmp/ipykernel_136663/905987518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_clinicalbert['processed_text'] = X_test_clinicalbert['processed_text'].fillna('missing')
/tmp/ipykernel_136663/905987518.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,icustay_id,slice_start,processed_text
0,200009,2189-11-30 10:34:32,missing
1,200009,2189-11-30 12:34:32,missing
2,200009,2189-11-30 14:34:32,missing
3,200009,2189-11-30 16:34:32,respiratory care note pt received intubated pl...
4,200009,2189-11-30 18:34:32,missing
...,...,...,...
316303,299998,2181-07-06 08:47:40,missing
316304,299998,2181-07-06 10:47:40,missing
316305,299998,2181-07-06 12:47:40,missing
316306,299998,2181-07-06 14:47:40,missing


# ClinicalBERT embedding

In [10]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("You're using: ", device)
# Load pre-trained Clinical BERT model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
Bio_ClinicalBERT_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Send the model to GPU if available
Bio_ClinicalBERT_model = Bio_ClinicalBERT_model.to(device)

# Function to generate embeddings with a progress bar
def get_clinical_bert_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Generating Embeddings"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = Bio_ClinicalBERT_model(**inputs)
        # Use the average of the last hidden state for sentence embedding
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(embedding.flatten().tolist())
    return embeddings

You're using:  cuda


In [11]:
X_train_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_train_clinicalbert['processed_text'])
X_test_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_test_clinicalbert['processed_text'])
X_val_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_val_clinicalbert['processed_text'])

Generating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 316308/316308 [1:27:19<00:00, 60.36it/s]
/tmp/ipykernel_136663/1993919717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_clinicalbert['clinical_bert_embeddings'] = get_clinical_bert_embeddings(X_train_clinicalbert['processed_text'])
Generating Embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105444/105444 [27:28<00:00, 63.96it/s]
/tmp/ipykernel_136663/1993919717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [12]:
# Save the datasets
X_train_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_train_clinicalbert.csv'), index=False)
X_test_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_test_clinicalbert.csv'), index=False)
X_val_clinicalbert.to_csv(os.path.join(output_folder_path, 'X_val_clinicalbert.csv'), index=False)

In [13]:
import pickle
# Saving X_train_bert
with open(os.path.join(output_folder_path, 'X_train_clinicalbert.pkl'), 'wb') as file:
    pickle.dump(X_train_clinicalbert, file)

# Saving X_test_bert
with open(os.path.join(output_folder_path, 'X_test_clinicalbert.pkl'), 'wb') as file:
    pickle.dump(X_test_clinicalbert, file)

# Saving X_val_bert
with open(os.path.join(output_folder_path, 'X_val_clinicalbert.pkl'), 'wb') as file:
    pickle.dump(X_val_clinicalbert, file)